In [2]:
import nltk
import os
import pandas as pd
import re
import nltk
import fsspec
import pickle
from azure.storage.blob import BlobClient
from pyspark.sql import SparkSession

StatementMeta(vksynapsespark, 19, 2, Finished, Available)

In [3]:
df = spark.read.load('abfss://tweetdata@dlsaviney.dfs.core.windows.net/tweets_temp.csv',format='csv',header=True)

StatementMeta(vksynapsespark, 19, 3, Finished, Available)

In [4]:
fsspec_obj = fsspec.open('abfss://tweetdata@dlsaviney.dfs.core.windows.net/logistic_model.pkl')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
with fsspec_obj.open() as file:
    bow_obj = pickle.load(file)
    model = pickle.load(file)

StatementMeta(vksynapsespark, 19, 4, Finished, Available)

[nltk_data] Downloading package stopwords to /home/trusted-service-
[nltk_data]     user/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /home/trusted-service-
[nltk_data]     user/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package omw-1.4 to /home/trusted-service-
[nltk_data]     user/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.
/home/trusted-service-user/cluster-env/clonedenv/lib/python3.8/site-packages/sklearn/base.py:313: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.21.3 when using version 0.22.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/home/trusted-service-user/cluster-env/clonedenv/lib/python3.8/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.logistic module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / fun

In [5]:
def tokenization(data):
    """
    :param data: It will receive the tweet and perform tokenization and remove the stopwords
    :return: It will return the cleaned data
    """
    stop_words = set(nltk.corpus.stopwords.words('english'))
    stop_words.remove('no')
    stop_words.remove('not')

    tokenizer = nltk.tokenize.TweetTokenizer()

    document = []
    for text in data:
        collection = []
        tokens = tokenizer.tokenize(text)
        for token in tokens:
            if token not in stop_words:
                if '#' in token:
                    collection.append(token)
                else:
                    collection.append(re.sub("@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+", " ", token))
        document.append(" ".join(collection))
    return document

StatementMeta(vksynapsespark, 19, 5, Finished, Available)

In [6]:
def lemmatization(data):
    """
    :param data: Receive the tokenized data
    :return: Return the cleaned data
    """
    lemma_function = nltk.stem.wordnet.WordNetLemmatizer()
    sentence = []
    for text in data:
        document = []
        words = text.split(' ')
        for word in words:
            document.append(lemma_function.lemmatize(word))
        sentence.append(" ".join(document))
    return sentence

def get_tweet_sentiment(my_tweet) -> str:
    """
        Here we'll perform predictions on the data given by the tweeter.
    """
    tokenized_data = tokenization([my_tweet])
    lemmatized_data = lemmatization(tokenized_data)
    temp = bow_obj.transform(lemmatized_data)
    pred = model.predict(temp)
    if pred == 0:
        return 'Positive'
    else:
        return 'Negative'

StatementMeta(vksynapsespark, 19, 6, Finished, Available)

In [14]:
# Converting Spark Dataframe to Pandas Dataframe
tweet_df = df.toPandas()
sentiment_results = []
for tweet in tweet_df['Tweet']:
    sentiment_results.append(get_tweet_sentiment(tweet))
tweet_df ['Sentiments_Predictions'] = sentiment_results
# Converting Pandas Dataframe into Spark Dataframe
spark_df =spark.createDataFrame(tweet_df)
# Create a temporary table view on the dataframe
spark_df.createOrReplaceTempView("sentiment_data")
# Repartition used For Storing Spark Dataframe into single parquet file
df2 = spark_df.repartition(1)
# Loading Dataframe to parquet file in primary ADLS by giving directory and folder name
df2.write.mode("append").parquet('/data1/parquetfolder')

StatementMeta(vksynapsespark, 19, 14, Finished, Available)

/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:289: UserWarning: createDataFrame attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  'JavaPackage' object is not callable
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)


In [8]:
%%spark
// Making Scala Dataframe from Temporary table
val scala_df = spark.sql("SELECT * FROM sentiment_data")
// For Loading Spark Dataframe into Synapse SQL Table by append mode
scala_df.write.mode("append").synapsesql("asa_twitter_db.dbo.tbl_tweet_sentiment")

StatementMeta(vksynapsespark, 19, 8, Finished, Available)

In [15]:
%%pyspark
df = spark.read.load('abfss://dlfsviney@dlsaviney.dfs.core.windows.net/data1/parquetfolder/part-00000-b87ae188-5159-413c-9a0e-a4daa795a1cb-c000.snappy.parquet', format='parquet')
display(df.limit(10))

StatementMeta(vksynapsespark, 19, 15, Finished, Available)

SynapseWidget(Synapse.DataFrame, 74122644-740f-4be9-8028-4ff806f29f0a)